In [2]:
import Base.Threads.@spawn

using NPZ

include("WGR-MCMClib/WGR.jl")
using .WGR

include("WGR-MCMClib/Alfheim.jl")
using .Alfheim

include("WGR-GAlib/Species.jl")
using .AlfheimG

import Base.flush
using Distributions

using Formatting
using Test
using Random

# buf = WGR.BlockBuffer(32 * Threads.nthreads())
buf = WGR.BlockBuffer(4096)
setBlocks(x,y,z,b) = WGR.setBlocks!(buf, x, y, z, b)
flush() = WGR.flush!(buf)
refreshNd(a,p,f = false) = WGR.refreshNd!(buf,a,p,f)

refreshNd (generic function with 2 methods)

In [3]:
mapsize = 32
mapheight = 16
bRep = [0x00000000, 0xBA8D65FF, 0x3155A6FF, 0x96C78CFF]

4-element Vector{UInt32}:
 0x00000000
 0xba8d65ff
 0x3155a6ff
 0x96c78cff

In [4]:
###### Initialization

Nb = length(bRep)
padding = 2

_map = ones(AlfheimG.Block, mapsize + 2*padding, mapheight + 2*padding, mapsize + 2*padding)
_map[1+padding:mapsize+padding, 1:padding, 1+padding:mapsize+padding] .= 0x02

############# Create Paths energy

# Costs
costs = zeros(Int, Nb, Nb, Nb, Nb, 6)

costs[1:Nb, 1:Nb, 1, 1, 1:6] .= 10 # -> Void
costs[1:Nb, 1:Nb, 2:Nb, 1:Nb, 1:6] .= 10 # -> Solid

costs[1:Nb, 1:Nb, 1, 2:Nb, 1:2] .= 10 # -> Road, vertical (should not appear)
costs[1:Nb, 1:Nb, 1, 2:Nb, 3:6] .= 2 # -> Road, horizontal

costs[1, 2:Nb, 1, 1, 1:6] .= 3 # Road -> Void (stairs)
costs[1, 1, 1, 2:Nb, 1:6] .= 3 # Void -> Road (stairs)

# Test cost
# costs = zeros(Int, Nb, Nb, 6)
# costs[1:Nb, 1:Nb, 1:6] .= 1

@show findall(x -> x>0, costs .== 0)

# Arrays
dist = fill(Int(2^30), size(_map))
dirc = ones(UInt8, size(_map)...)

# Entrance
entranceSize = mapsize ÷ 4
entranceStart = (mapsize + 2 * padding - entranceSize) ÷ 2 + 1
entranceEnd = entranceStart + entranceSize

# 4 Entries
starts = []

dist[entranceStart:entranceEnd, 1:1 + padding, 1:padding] .= 0
starts = [starts; [[x, 1 + padding, padding] for x in entranceStart:entranceEnd]]
_map[entranceStart:entranceEnd, 1:padding, 1:padding] .= 3

dist[1:padding, 1:1 + padding, entranceStart:entranceEnd] .= 0
starts = [starts; [[padding, 1 + padding, x] for x in entranceStart:entranceEnd]]
_map[1:padding, 1:padding, entranceStart:entranceEnd] .= 3

dist[entranceStart:entranceEnd, 1:1 + padding, end:-1:end-padding+1] .= 0
starts = [starts; [[x, 1 + padding, size(_map)[3]-padding+1] for x in entranceStart:entranceEnd]]
_map[entranceStart:entranceEnd, 1:padding, end:-1:end-padding+1] .= 3

dist[end:-1:end-padding+1, 1:1 + padding, entranceStart:entranceEnd] .= 0
starts = [starts; [[size(_map)[1]-padding+1, 1 + padding, x] for x in entranceStart:entranceEnd]]
_map[end:-1:end-padding+1, 1:padding, entranceStart:entranceEnd] .= 3

@show [1, 1, 1] .+ padding

########### Create Map

testmap = AlfheimG.Map(
    _map,
    [1 + padding, 1 + padding, 1 + padding],
    [mapsize + padding, mapheight + padding, mapsize + padding]
)

########### Create objectives

pathsEnergy = Alfheim.Paths(costs, starts, [[0,0,0], [0,-1,0]], testmap.min, testmap.max, dist, dirc)
pathsWeight = 0.07

blockMarginalsE = Alfheim.BlockPropotion(0.7, AlfheimG.AirBlock) # Target: 65% Air remains
blockMarginalsW = mapsize^2 * mapheight * 6

# Energies
Etotal = Alfheim.VoxelGibbsDistribution(
    [
        blockMarginalsE,
#         pathsEnergy
    ],
    [
        blockMarginalsW,
#         pathsWeight
    ]
)

findall((x->begin
            #= In[4]:27 =#
            x > 0
        end), costs .== 0) = CartesianIndex{5}[]
[1, 1, 1] .+ padding = [3, 3, 3]


Main.Alfheim.VoxelGibbsDistribution(Main.Alfheim.AbstractEnergy[Main.Alfheim.BlockPropotion(0.7, 0x01, 0, Main.Alfheim.L2, Main.Alfheim.BlockPropotionState(0, 0.0), Main.Alfheim.BlockPropotionState[])], [98304.0], [0.0], [0.0])

In [5]:
# g = [2, 0, 5, 5, 1, 0, 4, 5, 1, 3, 5, 5, 2, 137, 8, 10]
g = Base.rand(UInt8, 16384)

city = AlfheimG.TestCity(g)

# refreshNd(bRep[testmap.map], [0, 64, 0], true)

Main.AlfheimG.TestCity{UInt8}(UInt8[0x22, 0xa4, 0x3e, 0xca, 0x9b, 0x9e, 0x11, 0xfb, 0x61, 0xe9  …  0x90, 0x4d, 0x60, 0xf8, 0x2f, 0xe7, 0x37, 0x58, 0x69, 0x55])

In [6]:
# using StatProfilerHTML

# Energy calculation
function fit(instance::AlfheimG.Specie, inmap::AlfheimG.Map, E::Alfheim.VoxelGibbsDistribution)
    copiedmap = deepcopy(inmap)
    AlfheimG.populate!(instance, copiedmap)
    dummyState = Alfheim.createMCMCState(copiedmap.map, deepcopy(E), Alfheim.DummyWalker(copiedmap.min, copiedmap.max))
    # refreshNd(bRep[copiedmap.map], [0, 64, 0], true)
    return - dummyState.p(dummyState)
end

@time fit(city, testmap, Etotal)

  0.934325 seconds (3.23 M allocations: 167.021 MiB, 3.75% gc time, 99.37% compilation time)


-14665.95555555556

In [ ]:
using Base.Threads
using StatsBase

# Simple GA
nPool = 32#Threads.nthreads() * 2
nParents = convert(Int64, nPool * 0.25)

collectedParentIdxs = collect(1:nParents)

geneLenth = 4096
pool = [AlfheimG.TestCity(Base.rand(UInt8, geneLenth)) for i in 1:nPool]
fitness = zeros(nPool)

totalIters = 1000
printInterval = 3
visInterval = 6

for iter in 1:(totalIters+1)
    
    @threads for ix in 1:nPool
        fitness[ix] = fit(pool[ix], testmap, Etotal)
    end
    
    # Choose top parents
    order = sortperm(fitness)
    parentsIdx = order[end-nParents+1:end]
    
    # Choose parents
    s1 = pool[parentsIdx[StatsBase.sample(collectedParentIdxs, nPool)]]
    s2 = pool[parentsIdx[StatsBase.sample(collectedParentIdxs, nPool)]]
    
    # Create offsprings
    pool = [AlfheimG.offspring(s1[i], s2[i], 0.01) for i in 1:nPool]
    AlfheimG.mutate!.(pool, 0.02)
    
    if iter % printInterval == 1
        printfmt("Iter {:4d} - fit: {:12.3f} ± {:<12.3f} ", iter, mean(fitness), std(fitness))
        flush(stdout)
        
        if iter % visInterval == 1
            print("(Visualizing)")
            flush(stdout)
            row = convert(Int64, floor(sqrt(nPool)))
            for ix in 1:length(pool)
                copiedmap = deepcopy(testmap)
                AlfheimG.populate!(pool[ix], copiedmap)
                refreshNd(
                    bRep[copiedmap.map], 
                    [
                        200 +    ((ix-1) % row) * (mapsize + 2*padding + 2), 
                        65, 
                        200 + div((ix-1),  row) * (mapsize + 2*padding + 2)
                    ], 
                    true
                )
            end
        end
        
        print("\n")
    end
end

Iter    1 - fit:   -12764.444 ± 2963.466     (Visualizing)
Iter    4 - fit:    -5977.481 ± 1224.982     
Iter    7 - fit:    -4774.044 ± 1067.279     (Visualizing)
Iter   10 - fit:    -3435.141 ± 1671.231     
Iter   13 - fit:    -2560.830 ± 1329.701     (Visualizing)
Iter   16 - fit:    -1450.904 ± 1307.755     
Iter   19 - fit:    -1622.874 ± 1857.947     (Visualizing)
Iter   22 - fit:    -1577.363 ± 1070.899     
Iter   25 - fit:    -1617.778 ± 1252.452     (Visualizing)
Iter   28 - fit:    -1736.533 ± 1598.007     
Iter   31 - fit:    -2067.437 ± 1424.723     (Visualizing)
Iter   34 - fit:    -1545.956 ± 1188.681     
Iter   37 - fit:    -1698.607 ± 1300.692     (Visualizing)